#### Description
This notebook examines the percent increase in CPIS area by region and the percent of irrigated land CPIS comprise by region. It now utilizes utility functions for configuration, path resolution, and country ISO codes, and includes improved code comments and markdown explanations.

## 1. Import Required Libraries and Utilities
Import all necessary libraries and utility functions for configuration, path resolution, and country ISO codes.

In [1]:
# Import standard libraries and utility functions
import pandas as pd
import matplotlib.pyplot as plt
import sys
import os

# Add the project root to sys.path so we can import from Code.utils everywhere
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from Code.utils.utility import load_config, resolve_path, regions_dict, region_names

# Load configuration
config = load_config()

# Helper function to read CSVs using config keys
read = lambda config_str: pd.read_csv(resolve_path(config[config_str]))

## 3. Load and Group Data by Region
Read CPIS and total irrigation area data, then group and sum by region.

In [ ]:
# Load CPIS and total irrigation area data for 2000 and 2021
CPIS_area_2000 = read('CPIS_Area_by_Country_2000_csv_path')
CPIS_area_2021 = read('CPIS_Area_by_Country_2021_csv_path')
total_area_2000 = read('AQUA_AfricaIrrigation_2000')
total_area_2021 = read('AQUA_AfricaIrrigation_2021')

# Initialize dictionary to store grouped data
region_data = {region: {'CPIS_2000': 0, 'CPIS_2021': 0, 'total_2000': 0, 'total_2021': 0} for region in region_names}

# Group and sum data for each region
for region, isos in regions_dict.items():
    region_data[region]['CPIS_2000'] = CPIS_area_2000[CPIS_area_2000['ISO'].isin(isos)]['Area_1000ha'].sum()
    region_data[region]['CPIS_2021'] = CPIS_area_2021[CPIS_area_2021['ISO'].isin(isos)]['Area_1000ha'].sum()
    region_data[region]['total_2000'] = total_area_2000[total_area_2000['ISO'].isin(isos)]['Value'].sum()
    region_data[region]['total_2021'] = total_area_2021[total_area_2021['ISO'].isin(isos)]['Value'].sum()

## 4. Calculate Percent Increase in CPIS and Irrigated Area
Compute the percent increase in CPIS and total irrigated area for each region.

In [ ]:
# Calculate percent increases for each region
for region in region_names:
    d = region_data[region]
    d['percent_increase_CPIS'] = 100 * (d['CPIS_2021'] - d['CPIS_2000']) / d['CPIS_2000'] if d['CPIS_2000'] else 0
    d['percent_increase_total'] = 100 * (d['total_2021'] - d['total_2000']) / d['total_2000'] if d['total_2000'] else 0

# Print summary of percent increases
for region in region_names:
    d = region_data[region]
    print(f"{region}: CPIS {'increased' if d['percent_increase_CPIS'] >= 0 else 'decreased'} by {abs(d['percent_increase_CPIS']):.2f}%")
    print(f"{region}: Total irrigation area {'increased' if d['percent_increase_total'] >= 0 else 'decreased'} by {abs(d['percent_increase_total']):.2f}%\n")

## 5. Compare CPIS as a Percentage of Irrigated Land
Calculate and visualize the share of irrigated land that is CPIS for each region and year.

In [ ]:
# Calculate CPIS as a percentage of total irrigated area for each region and year
ratios = {region: {'2000': 0, '2021': 0} for region_names}
for region in region_names:
    d = region_data[region]
    ratios[region]['2000'] = (d['CPIS_2000'] / d['total_2000'] * 100) if d['total_2000'] else 0
    ratios[region]['2021'] = (d['CPIS_2021'] / d['total_2021'] * 100) if d['total_2021'] else 0

# Plot CPIS and total irrigated area for each region
fig, axs = plt.subplots(2, 2, figsize=(14, 10))
bar_width = 0.35
colors = ['#1f77b4', '#ff7f0e']
plot_regions = region_names[:4]  # Only plot the first four regions for 2x2 grid

for i, region in enumerate(plot_regions):
    ax = axs[i // 2, i % 2]
    CPIS_vals = [region_data[region]['CPIS_2000'], region_data[region]['CPIS_2021']]
    total_vals = [region_data[region]['total_2000'], region_data[region]['total_2021']]
    index = [0, 1]
    ax.bar([x - bar_width for x in index], CPIS_vals, bar_width, label='CPIS', color=colors[0])
    ax.bar(index, total_vals, bar_width, label='Total', color=colors[1])
    ax.set_title(f'Irrigation Area in {region}', fontsize=14)
    ax.set_ylabel('Area (1000ha)', fontsize=12)
    ax.set_xlabel('Year', fontsize=12)
    ax.set_xticks(index)
    ax.set_xticklabels(['2000', '2021'], fontsize=12)
    ax.legend(fontsize=12)
    ax.grid(True, linestyle='--', alpha=0.7)
    # Annotate with CPIS share
    ratio_2000 = f"In 2000, CPIS comprised {ratios[region]['2000']:.2f}% of irrigated area."
    ratio_2021 = f"In 2021, CPIS comprised {ratios[region]['2021']:.2f}% of irrigated area."
    ax.text(0.5, -0.15, f"{ratio_2000}\n{ratio_2021}", ha='center', va='top', transform=ax.transAxes, fontsize=10)

plt.tight_layout(pad=2)
plt.savefig(resolve_path(config['2_Figure1']))
plt.show()